In [1]:
import numpy as np
import pandas as pd

In [2]:
url = 'https://github.com/mattharrison/datasets/raw/master/data/2020-jetbrains-python-survey.csv'

In [3]:
jb = pd.read_csv(url)

/var/folders/0m/djy5jf494nl4fkspl_602jb00000gn/T/ipykernel_798/1530167600.py:1: DtypeWarning: Columns (152) have mixed types. Specify dtype option on import or set low_memory=False.
  jb = pd.read_csv(url)


In [4]:
import collections

In [5]:
counter = collections.defaultdict(list)

In [6]:
for col in sorted(jb.columns):
    period_count = col.count('.')
    if period_count >= 2:
        part_end = 2
    else:
        part_end = 1
    parts = col.split('.')[:part_end]
    counter['.'.join(parts)].append(col)

In [7]:
uniq_cols = []

In [8]:
for cols in counter.values():
    if len(cols) == 1:
        uniq_cols.extend(cols)

In [9]:
jb2 = (jb
 [uniq_cols]
 .rename(columns=lambda c: c.replace('.', '_'))
 .assign(age=lambda df_:df_.age.str.slice(0,2).astype(float)
             .astype('Int64'),
         are_you_datascientist=lambda df_:df_.are_you_datascientist
             .replace({'Yes': True, 'No': False, np.nan: False}),
         company_size=lambda df_:df_.company_size.replace({
             'Just me': 1, 'Not sure': np.nan, 
             'More than 5,000': 5000, '2–10': 2, '11–50':11,
             '51–500': 51, '501–1,000':501,
             '1,001–5,000':1001}).astype('Int64'),
         country_live=lambda df_:df_.country_live.astype('category'),
         employment_status=lambda df_:df_.employment_status
              .fillna('Other').astype('category'),
         is_python_main=lambda df_:df_.is_python_main
              .astype('category'),
         team_size=lambda df_:df_.team_size
             .str.split(r'-', n=1, expand=True)
             .iloc[:,0].replace('More than 40 people', 41)
             .where(df_.company_size!=1, 1).astype(float),
         years_of_coding=lambda df_:df_.years_of_coding
             .replace('Less than 1 year', .5).str.extract(r'(\d+)')
             .astype(float),
         python_years=lambda df_:df_.python_years
             .replace('Less than 1 year', .5).str.extract(r'(\d+)')
             .astype(float),
         python3_ver=lambda df_:df_.python3_version_most
              .str.replace('_', '.').str.extract(r'(\d\.\d)')
              .astype(float),
         use_python_most=lambda df_:df_.use_python_most
              .fillna('Unknown')
        )
    .drop(columns=['python2_version_most'])
)

In [10]:
jb2

,age,are_you_datascientist,company_size,country_live,employment_status,first_learn_about_main_ide,how_often_use_main_ide,ide_main,is_python_main,job_team,main_purposes,missing_features_main_ide,nps_main_ide,python_years,python3_version_most,several_projects,team_size,use_python_most,years_of_coding,python3_ver
0,30,False,1,NaN,Partially employed by a company / organization,Conference / User Group,Weekly,PyCharm Community Edition,Yes,Work as an external consultant or trainer,For work,"No, it has all the features I need",3.0,3.0,Python 3_7,"Yes, I work on many different projects",1.0,Unknown,1.0,3.7
1,21,True,5000,India,Fully employed by a company / organization,School / University,Daily,VS Code,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",8.0,3.0,Python 3_6,"Yes, I work on one main and several side projects",2.0,Software prototyping,3.0,3.6
2,30,False,5000,United States,Fully employed by a company / organization,Friend / Colleague,Daily,Vim,Yes,Work on your own project(s) independently,Both for work and personal,"No, it has all the features I need",10.0,3.0,Python 3_6,"Yes, I work on one main and several side projects",NaN,DevOps / System administration / Writing autom...,3.0,3.6
3,<NA>,False,<NA>,NaN,Other,Friend / Colleague,Daily,PyCharm Professional Edition,Yes,NaN,Both for work and personal,Yes – Please list:,10.0,11.0,Python 3_8,"Yes, I work on many different projects",1.0,Web development,11.0,3.8
4,21,False,<NA>,Italy,Student,Search engines,Daily,VS Code,Yes,Work on your own project(s) independently,"For personal, educational or side projects","No, it has all the features I need",10.0,1.0,Python 3_8,"Yes, I work on one main and several side projects",1.0,Web development,NaN,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54457,21,False,2,Russian Federation,Fully employed by a company / organization,School / University,Daily,Vim,Yes,Work on your own project(s) independently,Both for work and personal,"No, it has all the features I need",10.0,6.0,Python 3_6,"Yes, I work on many different projects",NaN,Data analysis,1.0,3.6
54458,<NA>,False,<NA>,NaN,Other,NaN,NaN,NaN,Yes,NaN,Both for work and personal,NaN,NaN,3.0,Python 3_7,NaN,1.0,Web development,1.0,3.7
54459,21,False,1,Russian Federation,Self-employed (a person earning income directl...,Friend / Colleague,Daily,PyCharm Professional Edition,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",10.0,3.0,Python 3_7,"Yes, I work on many different projects",1.0,Web development,6.0,3.7
54460,30,True,51,Spain,Fully employed by a company / organization,Search engines,Daily,Other,Yes,Work on your own project(s) independently,Both for work and personal,Yes – Please list:,3.0,6.0,Python 3_7,"Yes, I work on many different projects",NaN,Data analysis,3.0,3.7


In [11]:
(jb2
    .pivot_table(index='country_live', columns='employment_status',
                values='age', aggfunc='mean')
)

TypeError: Int64

In [ ]:
pd.crosstab(index=jb2.country_live, columns=jb2.employment_status,
           values=jb2.age, aggfunc='mean')

In [ ]:
(jb2
    .groupby(['country_live', 'employment_status'])
    .age
    .mean()
    .unstack()
)

In [ ]:
def per_emacs(ser):
    return ser.str.contains('Emacs').sum() / len(ser) * 100

In [ ]:
(jb2
    .pivot_table(index='country_live', values='ide_main', aggfunc=per_emacs)
)

In [ ]:
pd.crosstab(index=jb2.country_live,
           columns=jb2.assign(iden='emacs_per').iden,
           values=jb2.ide_main, aggfunc=per_emacs)

In [ ]:
(jb2
    .groupby('country_live')
    [['ide_main']]
    .agg(per_emacs)
)

In [12]:
(jb2
    .pivot_table(index='country_live', values='age', aggfunc=(min, max))
)

,max,min
country_live,,
Algeria,60,18
Argentina,60,18
Armenia,60,18
Australia,60,18
Austria,60,18
...,...,...
United States,60,18
Uruguay,60,18
Uzbekistan,60,18


In [13]:
(jb2
    .groupby('country_live')
    .age
    .agg([min, max])
)

,min,max
country_live,,
Algeria,18,60
Argentina,18,60
Armenia,18,60
Australia,18,60
Austria,18,60
...,...,...
United States,18,60
Uruguay,18,60
Uzbekistan,18,60


In [14]:
(jb2
    .pivot_table(index='country_live', aggfunc=(min, max))
)

/var/folders/0m/djy5jf494nl4fkspl_602jb00000gn/T/ipykernel_798/1627126626.py:1: FutureWarning: ['are_you_datascientist', 'employment_status', 'first_learn_about_main_ide', 'how_often_use_main_ide', 'ide_main', 'is_python_main', 'job_team', 'main_purposes', 'missing_features_main_ide', 'python3_version_most', 'several_projects'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  (jb2


age     company_size     nps_main_ide      python3_ver       \
              max min          max min          max  min         max  min   
country_live                                                                
Algeria        60  18         5000   1         10.0  0.0         3.9  3.5   
Argentina      60  18         5000   1         10.0  3.0         3.9  3.5   
Armenia        60  18         5000   1         10.0  5.0         3.9  3.5   
Australia      60  18         5000   1         10.0  0.0         3.9  3.5   
Austria        60  18         5000   1         10.0  3.0         3.9  3.5   
...            ..  ..          ...  ..          ...  ...         ...  ...   
United States  60  18         5000   1         10.0  0.0         3.9  3.5   
Uruguay        60  18         5000   2         10.0  5.0         3.9  3.5   
Uzbekistan     60  18         5000   1         10.0  2.0         3.9  3.6   
Venezuela      60  18         5000   1         10.0  1.0         3.9  3.5   
Viet Nam       60  18         5000   1         10.0  5.0         3.9  3.5   

              python_years      team_size       use_python_most  \
                       max  min       max  min              max   
country_live                                                      
Algeria               11.0  1.0      21.0  1.0  Web development   
Argentina             11.0  1.0      21.0  1.0  Web development   
Armenia                6.0  1.0      41.0  1.0  Web development   
Australia             11.0  1.0      41.0  1.0  Web development   
Austria               11.0  1.0      13.0  1.0  Web development   
...                    ...  ...       ...  ...              ...   
United States         11.0  1.0      41.0  1.0  Web development   
Uruguay               11.0  1.0      41.0  1.0  Web development   
Uzbekistan             6.0  1.0       2.0  1.0  Web development   
Venezuela             11.0  1.0      21.0  1.0  Web development   
Viet Nam              11.0  1.0      41.0  1.0  Web development   

                                 years_of_coding       
                             min             max  min  
country_live                                           
Algeria        Computer graphics            11.0  1.0  
Argentina          Data analysis            11.0  1.0  
Armenia        Computer graphics            11.0  1.0  
Australia      Computer graphics            11.0  1.0  
Austria        Computer graphics            11.0  1.0  
...                          ...             ...  ...  
United States  Computer graphics            11.0  1.0  
Uruguay            Data analysis            11.0  1.0  
Uzbekistan     Computer graphics             6.0  1.0  
Venezuela          Data analysis            11.0  1.0  
Viet Nam       Computer graphics            11.0  1.0  

[76 rows x 16 columns]

In [17]:
(jb2
    .groupby('country_live')
    .agg([min, max])
)

/var/folders/0m/djy5jf494nl4fkspl_602jb00000gn/T/ipykernel_798/2068018219.py:1: FutureWarning: ['are_you_datascientist', 'employment_status', 'first_learn_about_main_ide', 'how_often_use_main_ide', 'ide_main', 'is_python_main', 'job_team', 'main_purposes', 'missing_features_main_ide', 'python3_version_most', 'several_projects'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  (jb2


age     company_size       nps_main_ide       python_years  \
              min max          min   max          min   max          min   
country_live                                                               
Algeria        18  60            1  5000          0.0  10.0          1.0   
Argentina      18  60            1  5000          3.0  10.0          1.0   
Armenia        18  60            1  5000          5.0  10.0          1.0   
Australia      18  60            1  5000          0.0  10.0          1.0   
Austria        18  60            1  5000          3.0  10.0          1.0   
...            ..  ..          ...   ...          ...   ...          ...   
United States  18  60            1  5000          0.0  10.0          1.0   
Uruguay        18  60            2  5000          5.0  10.0          1.0   
Uzbekistan     18  60            1  5000          2.0  10.0          1.0   
Venezuela      18  60            1  5000          1.0  10.0          1.0   
Viet Nam       18  60            1  5000          5.0  10.0          1.0   

                    team_size          use_python_most                   \
                max       min   max                min              max   
country_live                                                              
Algeria        11.0       1.0  21.0  Computer graphics  Web development   
Argentina      11.0       1.0  21.0      Data analysis  Web development   
Armenia         6.0       1.0  41.0  Computer graphics  Web development   
Australia      11.0       1.0  41.0  Computer graphics  Web development   
Austria        11.0       1.0  13.0  Computer graphics  Web development   
...             ...       ...   ...                ...              ...   
United States  11.0       1.0  41.0  Computer graphics  Web development   
Uruguay        11.0       1.0  41.0      Data analysis  Web development   
Uzbekistan      6.0       1.0   2.0  Computer graphics  Web development   
Venezuela      11.0       1.0  21.0      Data analysis  Web development   
Viet Nam       11.0       1.0  41.0  Computer graphics  Web development   

              years_of_coding       python3_ver       
                          min   max         min  max  
country_live                                          
Algeria                   1.0  11.0         3.5  3.9  
Argentina                 1.0  11.0         3.5  3.9  
Armenia                   1.0  11.0         3.5  3.9  
Australia                 1.0  11.0         3.5  3.9  
Austria                   1.0  11.0         3.5  3.9  
...                       ...   ...         ...  ...  
United States             1.0  11.0         3.5  3.9  
Uruguay                   1.0  11.0         3.5  3.9  
Uzbekistan                1.0   6.0         3.6  3.9  
Venezuela                 1.0  11.0         3.5  3.9  
Viet Nam                  1.0  11.0         3.5  3.9  

[76 rows x 16 columns]

In [18]:
(jb2
    .pivot_table(index='country_live',
                aggfunc={'age':['min', 'max'],
                        'team_size':'mean'})
)

age     team_size
              max min      mean
country_live                   
Algeria        60  18  2.428571
Argentina      60  18  3.192053
Armenia        60  18  6.076923
Australia      60  18  2.710884
Austria        60  18  2.448000
...            ..  ..       ...
United States  60  18  3.391337
Uruguay        60  18  4.692308
Uzbekistan     60  18  1.160000
Venezuela      60  18  1.812500
Viet Nam       60  18  3.285714

[76 rows x 3 columns]

In [19]:
(jb2
    .groupby('country_live')
    .agg({'age':['min', 'max'],
         'team_size':'mean'})
)

age     team_size
              min max      mean
country_live                   
Algeria        18  60  2.428571
Argentina      18  60  3.192053
Armenia        18  60  6.076923
Australia      18  60  2.710884
Austria        18  60  2.448000
...            ..  ..       ...
United States  18  60  3.391337
Uruguay        18  60  4.692308
Uzbekistan     18  60  1.160000
Venezuela      18  60  1.812500
Viet Nam       18  60  3.285714

[76 rows x 3 columns]

In [20]:
(jb2
    .groupby('country_live')
    .agg(age_min=('age', min),
        age_max=('age', max),
        team_size_mean=('team_size', 'mean')
        )
)

,age_min,age_max,team_size_mean
country_live,,,
Algeria,18,60,2.428571
Argentina,18,60,3.192053
Armenia,18,60,6.076923
Australia,18,60,2.710884
Austria,18,60,2.448000
...,...,...,...
United States,18,60,3.391337
Uruguay,18,60,4.692308
Uzbekistan,18,60,1.160000


In [21]:
(jb2.pivot_table(index=['country_live', 'ide_main'],
                values='age', aggfunc=[min, max]))

min max
                                          age age
country_live ide_main                            
Algeria      Atom                          18  60
             Eclipse + Pydev               18  30
             IDLE                          18  50
             IntelliJ IDEA                 21  21
             Jupyter Notebook              21  30
...                                        ..  ..
Viet Nam     PyCharm Community Edition     21  30
             PyCharm Professional Edition  18  30
             Spyder                        21  21
             VS Code                       18  30
             Vim                           18  40

[1059 rows x 2 columns]

In [22]:
(jb2
    .groupby(by=['country_live', 'ide_main'])
    [['age']]
    .agg([min, max])
)

age      
                                                     min   max
country_live ide_main                                         
Algeria      Atom                                     18    60
             Eclipse + Pydev                          18    30
             Emacs                                  <NA>  <NA>
             IDLE                                     18    50
             IntelliJ IDEA                            21    21
...                                                  ...   ...
Viet Nam     Python Tools for Visual Studio (PTVS)  <NA>  <NA>
             Spyder                                   21    21
             Sublime Text                           <NA>  <NA>
             VS Code                                  18    30
             Vim                                      18    40

[1292 rows x 2 columns]

In [23]:
(jb2
    .groupby(by=['country_live', 'ide_main'])
    [['age']]
    .agg([min, max])
    .swaplevel(axis='columns')
)

min   max
                                                     age   age
country_live ide_main                                         
Algeria      Atom                                     18    60
             Eclipse + Pydev                          18    30
             Emacs                                  <NA>  <NA>
             IDLE                                     18    50
             IntelliJ IDEA                            21    21
...                                                  ...   ...
Viet Nam     Python Tools for Visual Studio (PTVS)  <NA>  <NA>
             Spyder                                   21    21
             Sublime Text                           <NA>  <NA>
             VS Code                                  18    30
             Vim                                      18    40

[1292 rows x 2 columns]

In [24]:
(jb2
    .groupby(by=['country_live', 'ide_main'])
    .agg(age_min=('age', min), age_max=('age', max))
)

age_min  age_max
country_live ide_main                                               
Algeria      Atom                                        18       60
             Eclipse + Pydev                             18       30
             Emacs                                     <NA>     <NA>
             IDLE                                        18       50
             IntelliJ IDEA                               21       21
...                                                     ...      ...
Viet Nam     Python Tools for Visual Studio (PTVS)     <NA>     <NA>
             Spyder                                      21       21
             Sublime Text                              <NA>     <NA>
             VS Code                                     18       30
             Vim                                         18       40

[1292 rows x 2 columns]

In [25]:
(jb2
    .groupby(by=['country_live', 'ide_main'], observed=True)
    .agg(age_min=('age', min), age_max=('age', max))
)

age_min  age_max
country_live       ide_main                                      
India              Atom                               18       40
                   Eclipse + Pydev                    18       60
                   Emacs                              21       50
                   IDLE                               18       60
                   IntelliJ IDEA                      18       30
...                                                  ...      ...
Dominican Republic PyCharm Community Edition          21       21
                   PyCharm Professional Edition       21       30
                   Sublime Text                       21       40
                   VS Code                            18       50
                   Vim                                21       21

[1059 rows x 2 columns]

In [26]:
def even_grouper(idx):
    return 'odd' if idx % 2 else 'even'

In [27]:
(jb2
    .groupby(even_grouper)
    .size()
)

even    27231
odd     27231
dtype: int64